In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#input_file = '/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/maize_upreg_1000.fasta'
input_file = '/content/drive/My Drive/Deep Learning with Python/maize_upreg_1000.fasta'
handler = open(input_file,'r')
list_string_stress=[]
sequence=""
count=0
start_flag=False;
for string in handler:
    if(string[0]=='>'):
        start_flag=True;
        length=len(sequence)
        if(sequence!="" and length>100):
            temp=0
            for char in sequence:
                if((char=='A' or char=='T' or char=='C' or char=='G')==False):
                    temp=1;
                    break
            if(temp==0):
                list_string_stress.append(sequence)
        sequence=""
        count+=1
        continue
    if(start_flag==True):
        sequence+=string[0:len(string)-1]
    
        
#input_file = '/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/maize_downreg_1000.fasta'
input_file = '/content/drive/My Drive/Deep Learning with Python/maize_downreg_1000.fasta'
handler = open(input_file,'r')
line=0;
sequence=""
count=0
start_flag=False;
for string in handler:
    if(string[0]=='>'):
        start_flag=True;
        length=len(sequence)
        if(sequence!="" and length>100):
            #print(length)
            temp=0
            for char in sequence:
                if((char=='A' or char=='T' or char=='C' or char=='G')==False):
                    temp=1;
                    break
            if(temp==0):
                list_string_stress.append(sequence)
        sequence=""
        count+=1
        continue
    if(start_flag==True):
        sequence+=string[0:len(string)-1]

    
        
stresslen= len(list_string_stress)
#input_file = '/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/maize_true_const_1000.fasta'
input_file = '/content/drive/My Drive/Deep Learning with Python/maize_true_const_1000.fasta'
handler = open(input_file,'r')
list_string_const=[]
line=0;
sequence=""
count=0
start_flag=False;
for string in handler:
    if(string[0]=='>'):
        start_flag=True;
        length=len(sequence)
        if(sequence!="" and length>100):
            #print(length)
            temp=0
            for char in sequence:
                if((char=='A' or char=='T' or char=='C' or char=='G')==False):
                    temp=1;
                    break
            if(temp==0):
                list_string_const.append(sequence)
        sequence=""
        count+=1
        continue
    if(start_flag==True):
        sequence+=string[0:len(string)-1]
#     if(len(list_string)==2*stresslen):
#         break;
constlen = len(list_string_const)        
print(constlen)
print(stresslen)



31532
9481


In [0]:
maxlen=1000
total =100000
import numpy as np
x_train = np.zeros((total,maxlen))
i=0
for string in list_string_stress:
  j=0
  for char in string:
    if(char=='A'):
      x_train[i][j]=0
    if(char=='T'):
      x_train[i][j]=1
    if(char=='C'):
      x_train[i][j]=2
    if(char=='G'):
      x_train[i][j]=3
    j=j+1
  i=i+1
i = 50000  
for string in list_string_const:
  j=0
  for char in string:
    if(char=='A'):
      x_train[i][j]=0
    if(char=='T'):
      x_train[i][j]=1
    if(char=='C'):
      x_train[i][j]=2
    if(char=='G'):
      x_train[i][j]=3
    j=j+1
  i=i+1

In [0]:
profile_stress = np.zeros((4,maxlen))
for row in x_train[0:stresslen]:
  j=0
  for col in row:
    profile_stress[int(col)][j]=profile_stress[int(col)][j]+1
    j=j+1
profile_const = np.zeros((4,maxlen))
for row in x_train[50000:constlen+50000]:
  j=0
  for col in row:
    profile_const[int(col)][j]=profile_const[int(col)][j]+1
    j=j+1


In [0]:
profile_stress =profile_stress/stresslen
profile_const=profile_const/constlen

In [0]:
import random

for j in range(0,1000):
  for i in [1,2,3]:
    profile_stress[i][j]=profile_stress[i-1][j]+profile_stress[i][j]
    
for i in range(stresslen,50000):
  for j in range(0,1000):
    x=random.random()
    k = 0 
    for y in profile_stress[:,j]:
      if y > x:
        break
      k+=1
    x_train[i][j]=k
    
for j in range(0,1000):
  for i in [1,2,3]:
    profile_const[i][j]=profile_const[i-1][j]+profile_const[i][j]
    
for i in range(constlen+50000,100000):
  for j in range(0,1000):
    x=random.random()
    k = 0 
    for y in profile_const[:,j]:
      if y > x:
        break
      k+=1
    x_train[i][j]=k
    

In [0]:
y_train = np.zeros((100000,))
for i in range(50000,100000):
  y_train[i]=1

x_train = x_train+1
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]
print(x_train.shape)
print(y_train.shape)

(100000, 1000)
(100000,)


In [0]:
import numpy as np
from keras import Input
from keras.models import Sequential,Model
from keras.layers import Embedding, Flatten, Dense,LSTM,MaxPooling1D,Conv1D,Dropout,merge,Permute,Lambda,Reshape,RepeatVector,Multiply,Activation
from keras import layers
from keras import backend as K
max_words = 5
embedding_dim = 4

units = 64

#functional model
embedding_matrix=np.array([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0]])
_input = Input(shape=[maxlen], dtype='int32')
embedded = Embedding(max_words,embedding_dim,input_length=maxlen,weights=[embedding_matrix],trainable=False)(_input)
#embedded = Reshape((250,units))(embedded)
conv1 = layers.Conv1D(16, 7, activation='relu')(embedded)
maxpool1 = layers.MaxPooling1D(2)(conv1)
conv2 = layers.Conv1D(32, 7, activation='relu')(maxpool1)
maxpool2 = layers.MaxPooling1D(2)(conv2)
conv3 = layers.Conv1D(64, 7, activation='relu')(maxpool2)
maxpool3 = layers.MaxPooling1D(2)(conv3)
lstm = LSTM(units,return_sequences=True)(maxpool3)

attention = Dense(1, activation='tanh')(lstm)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(units)(attention)
attention = Permute([2, 1])(attention)

sent_representation = Multiply()([lstm, attention])
sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units,))(sent_representation)

# flat = Flatten()(embedded)
# dense0 = Dense(256, activation='relu')(flat)
# drop0 = Dropout(0.3)(dense0)
# dense = Dense(128, activation='relu')(drop0)
# drop = Dropout(0.3)(dense)
# dense1 = Dense(64, activation='relu')(drop)
# drop1 = Dropout(0.3)(dense1)
# dense2 = Dense(32, activation='relu')(drop1)
# drop2 = Dropout(0.3)(dense2)
# output1 = Dense(1, activation='sigmoid')(drop2)
output1 = Dense(1, activation='sigmoid')(sent_representation)
model = Model(input=_input, output=output1)

#sequential model
# embedding_matrix=np.array([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0]])

# model = Sequential()
# model.add(Embedding(max_words, embedding_dim,input_length=maxlen))
# model.add(layers.Conv1D(16, 7, activation='relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(32, 7, activation='relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(64, 7, activation='relu'))
# model.add(LSTM(64))
# # model.add(LSTM(64,return_sequences=True))
# # model.add(layers.MaxPooling1D(3))
# # model.add(Flatten())
# # model.add(Dense(64, activation='relu'))
# # model.add(Dropout(0.3))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='sigmoid'))

# model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False


model.summary()

W0825 10:45:59.912473 140084182542208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1000, 4)      20          input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 994, 16)      464         embedding_5[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 497, 16)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=200,
                    batch_size=256,validation_split=.2)
#model.save('/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/my_model.h5')
#model.save('/content/drive/My Drive/Deep Learning with Python/my_model_19_8.h5')

NameError: ignored